In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import BitsAndBytesConfig
import torch
import json
import bitsandbytes
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
# from unsloth import FastLanguageModel
# unsloth doesn't work since unsloth need triton and triton isn't available on windows
import torch
from datasets import Dataset
import time

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24")

In [ ]:
# Create a BitsAndBytesConfig object with 8-bit loading
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Initialize an empty model with the correct configuration
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
        quantization_config=quant_config
    )

# Load the model with disk offloading
# model = load_checkpoint_and_dispatch(
#     model,
#     "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
#     device_map="auto",
#     offload_folder="D:\mtrie\OneDrive\Downloads\MLClub\offload"  # Specify the folder for disk offloading
# )

In [4]:
# Example input_ids (replace with actual input data)
input_text = "Hello llama model"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Ensure inputs are on the same device as the model
inputs = input_ids.to(next(model.parameters()).device)

# Generate outputs from the model
outputs = model.generate(inputs, max_new_tokens=100, use_cache=True)

# Batch decoding
text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(text)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


KeyboardInterrupt: 

In [ ]:
import time  # Import the time module

text = ""  # Initialize text before the loop

while True:
    input_text = input("Enter input text: ")
    if input_text.lower() == 'exit':
        print("Goodbye!")
        break
    text = input_text
    print(f"Enter input text: {input_text}")
    
    batch = []

    # batch is one, we will be serving one person only. 
    for k in range(1):
        batch.append(text)

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Ensure inputs are on the same device as the model
    inputs = input_ids.to(next(model.parameters()).device)
    start_time = time.time()
    # Generate outputs from the model
    outputs = model.generate(inputs, max_new_tokens=100, use_cache=True) # can set temperature 
    end_time = time.time()
    # Batch decoding
    text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    print(text)

    num_tokens = outputs.shape[-1]  # Number of tokens in the output
    time_taken = end_time - start_time
    tokens_per_second = num_tokens / time_taken

    last_eot = text.rfind("<|eot_id|>")

    # Find the second-to-last `[eot]` token (search up to the last one)
    second_last_eot = text.rfind("<|end_header_id|>")

    # Extract the content between them
    if second_last_eot != -1 and last_eot != -1:
        content_between = text[second_last_eot + len("<|end_header_id|>"):last_eot].strip()

        # THIS IS WHERE THE OUTPUT IS PRINTED TO THE TERMINAL
        print(content_between)
    else:
        print("Not enough [eot] tokens found.")
    print(f"----------\nTokens per second: {tokens_per_second}")
    final_time = time.time()
    difference = final_time - end_time
    print(f"\nPost Processing Time: {difference}\n")

Enter input text: hello world
